In [3]:
#교수님이 주신 데이터 전처리과정
import pandas as pd
import os
dirpath = './rawfiles/'
files = os.listdir(dirpath)

raw = pd.DataFrame()

for file in files:
    filepath = os.path.join(dirpath, file)
    
    # 폴더는 건너뛰기 (.ipynb_checkpoints 포함)
    if os.path.isdir(filepath):
        continue
        
    # CSV 파일만 처리
    if file.endswith('.csv'):
        try:
            temp = pd.read_csv(filepath)
            raw = pd.concat([raw, temp], axis=0)
        except Exception as e:
            print(f"읽기 실패: {filepath} / 오류: {e}")

raw = raw.reset_index(drop=True) 

raw['사용일자'] = pd.to_datetime(raw['사용일자'], format='%Y%m%d')

raw['요일'] = raw['사용일자'].dt.dayofweek

conv = {
    0 : '월',
    1 : '화',
    2 : '수',
    3 : '목',
    4 : '금',
    5 : '토',
    6 : '일'
}

raw['요일'] = raw['요일'].map(conv)

raw['승하자총승객수'] = raw['승차총승객수'] + raw['하차총승객수']

fpath = './subway_raw.csv'
raw.to_csv(fpath, index=False, encoding='utf-8-sig')

In [9]:
# 문제: 상명대학교 학생들에게 보다 더 가까운 지하철 역이 필요하다. 상명대학교 학생들은 상명대학교와 지하철이 너무 멀어서 지하철을 다른 학교에 비해 덜 이용한다.
# 가설: 상명대학교 학생들은 다른 학교에 비해 지하철을 적게 이용한다.
# 가설2: 대학교 인근 지하철역은 학기 중일 때와 방학 중일 때 승하차총승객수가 유의한 양의 차이가 있다.
# 가설에 이용한 공식: (1,2월(방학) 승하차총승객수 평균 - 3,4,5,6월(개강) 승하차총승객수 평균)/평균 승하차총승객수

from scipy import stats
import sys
station_name=input('알고자 하는 역을 입력하세요 (예시 경복궁역 입력: 경복궁):')
    
station= raw[raw['역명'] == station_name]
vacation_station=station[(station['사용일자'] >= '2019-01-01') & (station['사용일자'] <= '2019-02-28')]
termtime_station=station[(station['사용일자'] >= '2019-03-01') & (station['사용일자'] <= '2019-06-30')]
vacation_station=vacation_station.groupby('사용일자')['승하자총승객수'].mean()
termtime_station=termtime_station.groupby('사용일자')['승하자총승객수'].mean()

t_statistic, p_value = stats.ttest_ind(termtime_station, vacation_station, equal_var=False)
print("\n=== T-검정 결과 ===")
print(f"T-통계량 (T-statistic): {t_statistic:.4f}")
print(f"P-value (유의확률): {p_value:.15f}")
print("--------------------------------------")
if p_value < 0.05: 
    print(f"{p_value:.15f}가 유의수준 5%(0.05)미만이므로 귀무가설 기각: 대학교가 학기 중일 때와 방학 중일 때 {station_name}역의 승하차총승객수는 유의한 차이가 있다.")
else:
    print(f"{p_value:.15f}가 유의수준 5%(0.05)이상이므로 귀무가설 채택: 대학교가 학기 중일 때와 방학 중일 때 {station_name}역의 승하차총승객수는 유의한 차이가 없다.")
    print("따라서 학기 중일 때와 방학 중일 때 승하차총승객수 차이비율을 계산할 수 없다.")
    sys.exit()



print("\n=== 학기 중일 때와 방학 중일 때 승하차총승객수 차이비율 계산 ===")
termtime_station_mean=termtime_station.mean()
vacation_station_mean=vacation_station.mean()
difference=float((termtime_station_mean-vacation_station_mean)/((termtime_station_mean+vacation_station_mean)/2)*100)
print(f"대학교가 학기 중일 때 {station_name}역의 승하차총승객수 평균: {int(termtime_station_mean)}명/일")
print(f"대학교가 방학일 때 {station_name}역의 승하차총승객수 평균: {int(vacation_station_mean)}명/일")
print(f"{station_name}역의 대학생 증가 수 (개강시기 총승하차층객수 평균 - 방학시기 총승하차층객수 평균): {int(termtime_station_mean-vacation_station_mean)}명/일")
print(f"{station_name}역의 평균 승하차총승객수 대비 개강시기 대학생 증가 비율: {difference:.3f}%")

알고자 하는 역을 입력하세요 (예시 경복궁역 입력: 경복궁): 건대입구



=== T-검정 결과 ===
T-통계량 (T-statistic): 3.0336
P-value (유의확률): 0.003204463919126
--------------------------------------
0.003204463919126가 유의수준 5%(0.05)미만이므로 귀무가설 기각: 대학교가 학기 중일 때와 방학 중일 때 건대입구역의 승하차총승객수는 유의한 차이가 있다.

=== 학기 중일 때와 방학 중일 때 승하차총승객수 차이비율 계산 ===
대학교가 학기 중일 때 건대입구역의 승하차총승객수 평균: 65533명/일
대학교가 방학일 때 건대입구역의 승하차총승객수 평균: 60722명/일
건대입구역의 대학생 증가 수 (개강시기 총승하차층객수 평균 - 방학시기 총승하차층객수 평균): 4810명/일
건대입구역의 평균 승하차총승객수 대비 개강시기 대학생 증가 비율: 7.620%
